In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
_ = load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]


def get_llm_output(temperature, max_tokens, system_role, prompt,
                   frequency_penalty=0, presence_penalty=0, stop=None, llm_type="openai"):
    """
    Gọi LLM và trả về nội dung response. Nếu system_role là chuỗi rỗng,
    ta chỉ đưa message user, không đưa message system.
    """
    start_time = time.time()
    if llm_type == "openai":
        # Sử dụng OpenAI API
        client = OpenAI(
            api_key=api_key,
        )
        # Chỉ thêm system message khi system_role không rỗng
        messages = []
        if system_role and system_role.strip():
            messages.append({"role": "system", "content": system_role})
        messages.append({"role": "user", "content": prompt})

        completion = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
            stop=stop,
        )
        end_time = time.time()
        print('Finish generating response in', round((end_time - start_time), 2), 'seconds')
        return completion.choices[0].message.content

In [31]:
import pymupdf


def extract_text_from_pdfs(files):
    """Extract full text from a list of PDF files."""
    text = ""
    for f in files or []:
        with pymupdf.open(f) as pdf:
            text += "".join(page.get_text() for page in pdf)
    return text

In [32]:
text = extract_text_from_pdfs(['sample_resume.pdf'])

In [33]:
def structure_cv(cv_text):
    system_prompt = """
You are a professional resume/CV formatter. 
Reformat the candidate's resume text into a clean, modern structure with these sections:
1. Header: Name, Contact Info
2. Summary: 2–3 sentence career summary
3. Experience: List of jobs with role, company, dates, bullet achievements
4. Education: Degrees, institutions, years
5. Projects (if any): title, description, technologies used
6. Skills: grouped by category (e.g., Languages, Tools, Frameworks)
7. Certifications (if any)

Output the resume in Markdown format. Use consistent bullet points and headers.
"""
    # Chia nhỏ nếu CV quá dài (ở đây giả định cv_text < 4000 tokens)
    prompt = f"""
Raw Resume Text:
\"\"\"
{cv_text}
\"\"\"

Please reformat as described.
"""
    return get_llm_output(
        temperature=0.3,
        max_tokens=500,
        system_role=system_prompt,
        prompt=prompt
    )

In [34]:
aftext = structure_cv(text)

Finish generating response in 19.9 seconds


In [35]:
print(aftext)

```
# Farah Martin
Data Analyst  
Email: farahmartin@email.com  
Phone: (123) 456-7890  
Location: Brooklyn, NY  
LinkedIn: [Link](https://www.linkedin.com/in/farahmartin)

## Summary
Experienced Data Analyst with a strong background in Mathematics and Economics. Proven abilities in building data and reporting infrastructures, designing and implementing A/B experiments, and leading teams to drive business growth and profitability.

## Experience
**Data Analyst**  
Fountain House, New York, NY  
May 2018 - Present  
- Built out the data and reporting infrastructure from the ground up using Tableau and SQL to provide real-time insights into the product, marketing funnels, and business KPIs.
- Designed and implemented A/B experiments for products to improve the conversion rate by 19 basis points and reduce churn by 12 basis points.
- Implemented long-term pricing experiment that improved customer value by 25%.
- Built operational reporting in Tableau to find areas of improvement for contr

In [13]:
def structure_jd(jd_text):
    system_prompt = """
You are a professional job description formatter. 
Reformat the candidate's job description text into a clean, modern structure with these sections. Leave 'no information' for any section that is not applicable:
1. Header: Job Title, Company Name
2. Summary: 2–3 sentence overview of the role
3. Responsibilities: List of key responsibilities and tasks
4. Requirements: Skills and qualifications needed
5. Preferred Qualifications: Additional skills that are a plus
6. Benefits: Perks and benefits offered
7. Application Process: How to apply

Output the job description in Markdown format. Use consistent bullet points and headers.
"""
    # Chia nhỏ nếu JD quá dài (ở đây giả định jd_text < 4000 tokens)
    prompt = f"""
Raw Job Description Text:
\"\"\"
{jd_text}
\"\"\"

Please reformat as described.
"""
    return get_llm_output(
        temperature=0.3,
        max_tokens=1200,
        system_role=system_prompt,
        prompt=prompt
    )

In [16]:
jd = extract_text_from_pdfs(['sample_jd.pdf'])
afjdtext = structure_jd(jd)
print(afjdtext)


Finish generating response in 5.51 seconds
### Header: Data Analyst, [No Company Name]
### Summary
We are seeking a talented and detail-oriented Data Analyst to join our dynamic team, responsible for interpreting data, analyzing results, and providing actionable insights to drive informed decision-making. The Data Analyst will work closely with stakeholders to understand their data needs and present findings in a clear and concise manner. This role requires a strong analytical mindset and excellent communication skills.

### Responsibilities
* Extract, transform, and load (ETL) data from various sources
* Clean and preprocess data to ensure accuracy and consistency
* Develop scripts and workflows to automate data collection and processing tasks
* Perform exploratory data analysis to uncover trends, patterns, and anomalies
* Apply statistical and analytical techniques to derive insights from complex datasets
* Conduct hypothesis testing and predictive modeling to support business object

In [18]:
def generate_feedback(cv_text, jd_text):
    """
    Use LLM to generate structured, comprehensive feedback on the resume:
    - Đưa ra các tiêu chí đánh giá (Assessment Criteria)
    - Đánh giá từng mục (Review) dựa trên tiêu chí
    - Gợi ý cải tiến (Improvement Suggestions) để tăng tỉ lệ đậu
    """
    system_prompt = """
You are a senior career advisor and HR specialist. 
Your task is to evaluate the candidate’s resume against the Job Description, 
using the following structured framework:

1. Assessment Criteria:
   a. Relevance of Experience – How well past roles map to required responsibilities.
   b. Skills Match – Presence and prominence of key technical and soft skills.
   c. Projects - Relevance and impact of projects related to the JD.
   d. Achievement Impact – Use of quantifiable results and action verbs.
   e. Keywords & ATS Optimization – Inclusion of terminology from the JD.
   f. Clarity & Readability – Logical sectioning, bullet usage, concise language.
   g. Professional Branding – Strong summary, consistent formatting, contact info.

2. For each criterion:
   - Provide a **brief review**: highlight strengths and gaps.
   - Offer **2–3 short actionable suggestions** to improve that aspect of the resume.

3. Finally, summarize **top 3 priority actions** the candidate should take 
   to significantly increase their chance of being shortlisted.

Return your output in Markdown, with sections:
## Assessment
- **Relevance of Experience**  
  - Review: …  
  - Suggestions:
    1. …
    2. …

(Repeat for each criterion)

## Top 3 Priority Actions
1. …
2. …
3. …
"""

    user_prompt = f"""
Job Description:
\"\"\"
{jd_text}
\"\"\"

Candidate Resume:
\"\"\"
{cv_text}
\"\"\"
"""

    return get_llm_output(
        temperature=0.3,
        max_tokens=1200,
        system_role=system_prompt,
        prompt=user_prompt
    )

In [19]:
generated_feedback = generate_feedback(aftext, afjdtext)

KeyboardInterrupt: 

In [ ]:
print(generated_feedback)

## Assessment
### Relevance of Experience
- Review: The candidate's experience is highly relevant to the job description, with a strong background in data analysis, business intelligence, and related roles. The candidate has worked in various industries, including insurance and non-profit, and has experience in leading cross-functional teams and collaborating with executive teams.
- Suggestions:
  1. Emphasize the specific skills and tools used in previous roles, such as SQL, Python, and Tableau, to match the job requirements.
  2. Quantify the impact of previous projects, such as the 19 basis points improvement in conversion rate and 12 basis points reduction in churn, to demonstrate the value added to previous employers.
  3. Consider adding a brief description of the company and industry for each previous role to provide context for the accomplishments.

### Skills Match
- Review: The candidate's skills match the job requirements, with proficiency in data analysis tools and programm

## Assessment
### Relevance of Experience
- Review: The candidate's experience is highly relevant to the job description, with a strong background in data analysis, business intelligence, and related roles. The candidate has worked in various industries, including insurance and non-profit, and has experience in leading cross-functional teams and collaborating with executive teams.
- Suggestions:
  1. Emphasize the specific skills and tools used in previous roles, such as SQL, Python, and Tableau, to match the job requirements.
  2. Quantify the impact of previous projects, such as the 19 basis points improvement in conversion rate and 12 basis points reduction in churn, to demonstrate the value added to previous employers.
  3. Consider adding a brief description of the company and industry for each previous role to provide context for the accomplishments.

### Skills Match
- Review: The candidate's skills match the job requirements, with proficiency in data analysis tools and programming languages, such as SQL, Python, and Tableau. However, the candidate could benefit from highlighting additional skills, such as machine learning and data mining techniques.
- Suggestions:
  1. Expand the skills section to include additional technical skills, such as R, Julia, or other programming languages relevant to the job.
  2. Emphasize the candidate's experience with data visualization tools, such as Tableau, and highlight any certifications or training in these areas.
  3. Consider adding a section for "soft skills" to highlight the candidate's excellent communication and presentation abilities.

### Projects
- Review: The candidate's projects and accomplishments are highly relevant to the job description, with a focus on data-driven solutions, operational efficiency, and revenue growth. However, the candidate could benefit from providing more details about the projects, such as the methodologies used and the impact on the business.
- Suggestions:
  1. Provide more details about the projects, such as the goals, methodologies, and outcomes, to demonstrate the candidate's expertise and value added.
  2. Consider adding a separate section for "projects" or "case studies" to showcase the candidate's accomplishments and provide a clear overview of their experience.
  3. Emphasize the candidate's ability to work independently and collaboratively on projects, and highlight any experience with agile methodologies or project management tools.

### Achievement Impact
- Review: The candidate's achievements are quantified, with specific metrics and outcomes, such as the 19 basis points improvement in conversion rate and 12 basis points reduction in churn. However, the candidate could benefit from using more action verbs and emphasizing the impact on the business.
- Suggestions:
  1. Use more action verbs, such as "Improved," "Increased," and "Enhanced," to describe the candidate's achievements and emphasize the impact on the business.
  2. Quantify the impact of the candidate's achievements, such as the revenue growth or cost savings, to demonstrate the value added to previous employers.
  3. Consider adding a section for "achievements" or "accomplishments" to showcase the candidate's impact and provide a clear overview of their experience.

### Keywords & ATS Optimization
- Review: The candidate's resume includes relevant keywords, such as "data analysis," "business intelligence," and "SQL," but could benefit from additional keywords and phrases to match the job requirements.
- Suggestions:
  1. Incorporate additional keywords and phrases from the job description, such as "machine learning," "data mining," and "data visualization," to improve the candidate's visibility in applicant tracking systems (ATS).
  2. Use a keyword cloud or tag cloud to visualize the candidate's skills and expertise, and provide a clear overview of their experience.
  3. Consider adding a section for "technical skills" or "tools" to showcase the candidate's proficiency in specific software and programming languages.

### Clarity & Readability
- Review: The candidate's resume is well-organized, with clear section headings and concise bullet points. However, the candidate could benefit from using more white space and emphasizing the most important information.
- Suggestions:
  1. Use more white space to improve the readability of the resume, and emphasize the most important information, such as the candidate's achievements and skills.
  2. Consider using a more modern font, such as Arial or Calibri, to improve the overall appearance of the resume.
  3. Use bullet points and action verbs to describe the candidate's experience and achievements, and provide a clear overview of their skills and expertise.

### Professional Branding
- Review: The candidate's professional branding is strong, with a clear and concise summary, consistent formatting, and a professional online presence.
- Suggestions:
  1. Consider adding a professional summary or career objective statement to the top of the resume, to provide a clear overview of the candidate's experience and goals.
  2. Use a consistent format throughout the resume, with clear section headings and concise bullet points, to improve the overall appearance and readability.
  3. Emphasize the candidate's unique value proposition, such as their experience in leading cross-functional teams or collaborating with executive teams, to differentiate themselves from other candidates.

## Top 3 Priority Actions
1. **Emphasize relevant technical skills**: Incorporate additional keywords and phrases from the job description, such as "machine learning," "data mining," and "data visualization," to improve the candidate's visibility in applicant tracking systems (ATS) and match the job requirements.
2. **Quantify achievements and impact**: Use more action verbs and emphasize the impact on the business, such as the revenue growth or cost savings, to demonstrate the value added to previous employers and provide a clear overview of the candidate's experience.
3. **Use a more modern and concise format**: Use more white space, a modern font, and concise bullet points to improve the readability and overall appearance

# Test RAG demo

In [ ]:
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from src.variables import GPT_EMBEDDING_MODEL
from dotenv import load_dotenv
import os
import gradio as gr
from langchain.schema import Document

_ = load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
PERSIST_DIR = "./chroma_db"
# Initialize embedding model and LLM
embeddings = OpenAIEmbeddings(model=GPT_EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY)

def get_rag_configs():
    """Initialize and return RAG configurations."""
    # Embedding model for RAG
    embeddings = OpenAIEmbeddings(
        model=GPT_EMBEDDING_MODEL,
        openai_api_key=OPENAI_API_KEY
    )
    # Text splitter for chunking documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=150, 
                        length_function=len, separators=["\n\n", "\n", ".", " ", ""]
                    ) 
    return embeddings, text_splitter

def save_doc_to_vector_store(text: str, collection_name: str, file_name: str = None):
    """Chunk text and index into Chroma under a specific collection."""
    embeddings, text_splitter = get_rag_configs()
    chunks = text_splitter.split_text(text)
    metadatas = []
    for i, chunk in enumerate(chunks):
        md = {
            "source": collection_name,
            "chunk_id": i,
        }
        if file_name is not None:
            md["file_name"] = file_name
        metadatas.append(md)

    docs = [
        Document(page_content=chunks[i], metadata=metadatas[i])
        for i in range(len(chunks))
    ]
    vectordb = Chroma(
        collection_name=collection_name,
        embedding_function=embeddings,
        persist_directory=PERSIST_DIR
    )
    # Add texts with metadata
    vectordb.add_documents(documents=docs)
    return vectordb

vector_store = save_doc_to_vector_store(aftext, "saveCV", file_name="sample_resume.pdf")


In [ ]:
results = vector_store.similarity_search(
    "Experience of the candidate in the field of DA and ML",
    k=2,
)

In [ ]:
for res in results:
    # Markdown separator
    print("---\n")
    # Content section
    print(f"**Content:**\n\n{res.page_content}\n")
    # Metadata section
    print("**Metadata:**")
    for key, value in res.metadata.items():
        print(f"- **{key}**: {value}")
    print("\n")

---

**Content:**

### Education
* **B.S. in Mathematics and Economics**, University of Pittsburgh, September 2010 - April 2014, Pittsburgh, PA

### Skills
* **Languages**: Python (Pandas, Scikit-learn), SQL
* **Tools**: Excel, Google Sheets, Tableau, Google Analytics
* **Frameworks**: A/B Testing & Experimentation
* **Leadership**: Proven experience in leading teams and collaborating with executive teams

### Certifications
None listed.

**Metadata:**
- **file_name**: sample_resume.pdf
- **chunk_id**: 7
- **source**: saveCV


---

**Content:**

### Education
* **B.S. in Mathematics and Economics**, University of Pittsburgh, September 2010 - April 2014, Pittsburgh, PA

### Skills
* **Languages**: Python (Pandas, Scikit-learn), SQL
* **Tools**: Excel, Google Sheets, Tableau, Google Analytics
* **Frameworks**: A/B Testing & Experimentation
* **Leadership**: Proven experience in leading teams and collaborating with executive teams

### Certifications
None listed.

**Metadata:**
- **source

# Test gradio interface

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
_ = load_dotenv()
api_key = os.environ["NVIDIA_API_KEY"]


def get_llm_output(temperature, max_tokens, system_role, prompt,
                   frequency_penalty=0, presence_penalty=0, stop=None):
    """
    Gọi LLM và trả về nội dung response. Nếu system_role là chuỗi rỗng,
    ta chỉ đưa message user, không đưa message system.
    """
    start_time = time.time()
    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key=api_key,
    )
    # Chỉ thêm system message khi system_role không rỗng
    messages = []
    if system_role and system_role.strip():
        messages.append({"role": "system", "content": system_role})
    messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop,
    )
    end_time = time.time()
    print('Finish generating response in', round((end_time - start_time), 2), 'seconds')
    return completion.choices[0].message.content

In [ ]:
# Define a fixed system prompt for the interviewer persona
INTERVIEWER_SYSTEM_PROMPT = """
You are a professional technical interviewer. 
Your goal is to ask clear, relevant follow-up questions to evaluate the candidate's skills and experience.
Be concise, polite, and stay on topic.
"""

def interview(user_message, history):
    """
    user_message: str, câu vừa nói/vừa trả lời của ứng viên
    history: list of (str, str) tuples, danh sách các lượt (candidate, interviewer)
    
    Trả về: (new_history, new_history) để Gradio cập nhật Chatbot.
    """
    # 1. Build a single prompt that includes past conversation
    conversation = ""
    for u, b in history:
        conversation += f"Candidate: {u}\nInterviewer: {b}\n"
    conversation += f"Candidate: {user_message}\nInterviewer:"

    # 2. Call your LLM wrapper
    bot_reply = get_llm_output(
        temperature=0.7,
        max_tokens=500,
        system_role=INTERVIEWER_SYSTEM_PROMPT,
        prompt=conversation
    )

    # 3. Append to history
    new_history = history + [(user_message, bot_reply)]

    # Gradio Chatbot với Textbox thường dùng (history, history) làm outputs
    return new_history, new_history

In [ ]:
import os
import gradio as gr
from gtts import gTTS
from openai import OpenAI

# Initialize OpenAI client for Whisper
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def transcribe_audio(audio_path):
    """speech to text."""
    with open(audio_path, "rb") as f:
        resp = openai.audio.transcriptions.create(
            model="whisper-1",
            file=f
        )
    return getattr(resp, "text", "") or getattr(resp, "transcription", "")

def synthesize_speech(text, lang="en"):
    """text to speech."""
    tts = gTTS(text=text, lang=lang)
    out_path = "response.mp3"
    tts.save(out_path)
    return out_path

def voice_interaction(user_audio, history, q_idx):
    # Nếu user chưa nói gì
    if not user_audio:
        return history, None, history, q_idx

    # STT
    user_text = transcribe_audio(user_audio)
    print("DEBUG: user_text =", user_text)

    # Next question
    next_idx = q_idx + 1
    if next_idx < len(INITIAL_QUESTIONS):
        bot_text = INITIAL_QUESTIONS[next_idx]
    else:
        bot_text = "🎉 Thank you! This concludes our interview."

    # TTS cho câu hỏi kế tiếp
    bot_audio = synthesize_speech(bot_text)

    # Update history: add (user_answer, bot_question)
    new_history = history + [(user_text, bot_text)]

    return new_history, bot_audio, new_history, next_idx

INITIAL_QUESTIONS = [
    "1. Tell me about a data analysis project where you used SQL to extract insights. What challenges did you face?",
    "2. How have you used Python libraries like pandas or NumPy to clean and transform data?",
    "3. Describe a dashboard you built in Tableau (or another BI tool). What metrics did you highlight?",
    # … add as many as you like
]

In [ ]:
with gr.Blocks(theme=gr.themes.Ocean()) as demo:
    gr.Markdown("<h2 style='text-align: center;'>🎤 Voice Interview</h2>")

    # State: current question index (starts at 0)
    question_idx = gr.State(0)
    # State: chat history seeded with the first question
    initial = [("", INITIAL_QUESTIONS[0])]
    chat_history = gr.State(initial)

    # Chatbot UI showing transcript & questions
    chatbot = gr.Chatbot(value=initial, label="Interview Transcript")

    with gr.Row():
        
        # Right column: voice controls
        with gr.Column():
            gr.Markdown("**Record your answer**")
            audio_in = gr.Audio(sources="microphone", type="filepath", label="Record your answer")
            speak_btn = gr.Button("Send Voice", variant="primary", size="md")
            gr.Markdown("**Interviewer’s reply (audio)**")
            audio_out = gr.Audio(interactive=False, label="Response Audio", autoplay=True)
    demo.load(
        fn=lambda: synthesize_speech(INITIAL_QUESTIONS[0]),
        inputs=None,
        outputs=audio_out
    )
    # Hook up button: process voice, update chat, play next question
    speak_btn.click(
        fn=voice_interaction,
        inputs=[audio_in, chat_history, question_idx],
        outputs=[chatbot, audio_out, chat_history, question_idx]
    )

demo.launch()

/tmp/ipykernel_20507/1812502522.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value=initial, label="Interview Transcript")


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


DEBUG: user_text = Ok, first I will mention which tool I use to query the data and convert it to seratable and then I use it to analyze the data and send it to the data modeling process to build the various predictors.


In [73]:
from openai import OpenAI
from dotenv import load_dotenv
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import os
import time
_ = load_dotenv()
api_key_nvi = os.environ["NVIDIA_API_KEY"]

api_key = os.environ["OPENAI_API_KEY"]

class QuestionBank(BaseModel):
    questions: List[str] = Field(descriptions="List of interview questions")

def get_llm_output_with_parser(max_tokens, system_role, prompt,
                                output_model=QuestionBank, llm_type="nvidia"):
    """
    Gọi LLM và trả về nội dung response đã được phân tích cú pháp.
    """
    parser = PydanticOutputParser(pydantic_object=output_model)
    prompt_template = PromptTemplate(
        input_variables=["system_role", "prompt"],
        template= system_role + "\n" + prompt
    )

    if llm_type == "openai":
        llm = ChatOpenAI(temperature=0, max_tokens=max_tokens, model="gpt-4", api_key=api_key)
    else:
        llm = ChatNVIDIA(temperature=0, max_tokens=max_tokens, model="meta/llama-3.3-70b-instruct", api_key=api_key_nvi)
    chain = prompt_template | llm | parser
    response = chain.invoke({
        "system_role": system_role,
        "prompt": prompt
    })
    return response.questions

In [ ]:
import json
from typing import List
from pydantic import BaseModel, ValidationError

def generate_question_bank(cv_text: str, jd_text: str):
    
    system_prompt = """
    You are a senior HR specialist and career advisor. Your task is to generate an interview question bank for a candidate, based on their resume (CV) and the provided Job Description (JD).

    Instructions:
    - Produce between 5 and 8 questions.
    - Order the questions from easiest to most difficult.
    - Cover the following areas:
    1. Career goals
    2. Technical skills
    3. Soft skills
    4. Language skills
    5. Projects
    6. Work experience (with additional depth and focus)
    - Questions should be clear, concise, and tailored to evaluate the candidate’s fit for the role described in the JD.

    Only return the output as a JSON object with a single key "questions" mapping to an array of strings, without any additional text.
    """

    user_prompt = f"""
    Job Description:
    '''
    {jd_text}
    '''

    Candidate Resume:
    '''
    {cv_text}
    '''
    """

    # Call the LLM
    raw_output = get_llm_output_with_parser(
        max_tokens=500,
        system_role=system_prompt,
        prompt=user_prompt
    )

    return raw_output

In [75]:
question_bank = generate_question_bank(aftext, afjdtext)
print("Generated Question Bank:")
print(question_bank)

KeyboardInterrupt: 

In [69]:
question_bank

['Can you tell us about your career goals and how this role aligns with them?',
 'You have experience with SQL, Python, and Tableau. Can you share an instance where you used these tools to solve a complex problem?',
 'Can you describe a situation where your communication skills were crucial in explaining complex data findings to non-technical stakeholders?',
 'Your resume mentions that you have led a team of data analysts. Can you share your approach to team management and collaboration?',
 'You mentioned designing and implementing A/B experiments at Fountain House. Can you walk us through the process and the impact it had?',
 'In your role at Wavely, you built a logistic regression model to help the SEO team. Can you elaborate on the process and the outcome?',
 'You have worked as a Data Analyst at two different companies. Can you discuss the differences in your roles and how your experiences at each company have prepared you for this position?',
 'Given your experience and our job de